In [1]:
import pandas as pd
import numpy as np
import hjson

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from get_dataframes import GetDataframes
from tools import import_config, import_datasets
from scipy.sparse import hstack

pd.set_option('display.max_columns', None)

In [ ]:

# ! link poster tmdb = https://image.tmdb.org/t/p/w500/{ici le lien du fichier jpg}
# ! récuperer également via l'api le lien des posters pour afficher dans app

In [27]:
tmd = import_datasets("clean_datasets/tmdb_full.parquet", "parquet")
tmd1 = import_datasets("clean_datasets/movies_cleaned.parquet", "parquet")

2023-11-11 10:28:01 INFO     Parquet loaded ! Importing tmdb_full...
2023-11-11 10:28:02 INFO     Parquet loaded ! Importing movies_cleaned...


In [25]:
tmd[tmd["imdb_id"].str.contains("tt0057967")]

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
166548,False,/aDi4GqSYi4pcTvS7A9AuXLuhUVQ.jpg,678948,"[Comedy, Crime]",None,25866,tt0057967,fr,Le Corniaud,"In this Frenco-Italian gangster parody, a shop...",15.45,/wmxzMpuW8Eq7MmPHn6jLDU8Q9ft.jpg,"[ES, FR, IT]",1965-03-24,0,111,[fr],Released,The funniest fraud a fall-guy ever fell for!,The Sucker,False,7.38,565,"[Les Films Corona, Explorer Film '58]",[]


In [28]:
tmd1

,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,region,cuts
0,tt0035423,Kate & Leopold,2001,118,"[Comedy, Fantasy, Romance]",6.40,87897,en,Kate & Leopold,15.77,[US],76019048.00,"[en, fr, it]",Released,FR,2000-2009
1,tt0052832,The Fugitive Kind,1960,119,"[Drama, Romance]",7.10,6914,en,The Fugitive Kind,5.98,[US],0.00,[en],Released,FR,1960-1969
2,tt0052997,The League of Gentlemen,1960,116,"[Comedy, Crime, Drama]",7.20,5105,en,The League of Gentlemen,5.16,[GB],0.00,[en],Released,FR,1960-1969
3,tt0053459,Eyes Without a Face,1960,90,"[Drama, Horror]",7.60,34278,fr,Les Yeux sans visage,9.05,"[FR, IT]",58656.00,[fr],Released,FR,1960-1969
4,tt0053472,Breathless,1960,90,"[Crime, Drama]",7.70,86230,fr,À bout de souffle,13.45,[FR],0.00,"[fr, en]",Released,FR,1960-1969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8287,tt9873892,They Cloned Tyrone,2023,122,"[Comedy, Mystery, Sci-Fi]",6.60,33197,en,They Cloned Tyrone,25.08,[US],0.00,[en],Released,FR,>2020
8289,tt9883996,Dream Horse,2020,113,"[Biography, Comedy, Drama]",6.90,6589,en,Dream Horse,12.54,"[GB, US]",6435260.00,[en],Released,FR,>2020
8290,tt9893250,I Care a Lot,2020,118,"[Comedy, Crime, Drama]",6.40,141372,en,I Care a Lot,22.05,"[GB, US]",0.00,[en],Released,FR,>2020
8291,tt9900782,Kaithi,2019,145,"[Action, Adventure, Crime]",8.50,39546,ta,கைதி,9.74,[IN],0.00,"[ta, te]",Released,FR,2010-2019


In [29]:
tmd1[tmd1["titre_id"].str.contains("tt0057967")]


,titre_id,titre_str,titre_date_sortie,titre_duree,titre_genres,rating_avg,rating_votes,original_language,original_title,popularity,production_countries,revenue,spoken_languages,status,region,cuts
187,tt0057967,The Sucker,1965,111,"[Adventure, Comedy, Crime]",7.30,7800,fr,Le Corniaud,15.45,"[ES, FR, IT]",0.00,[fr],Released,FR,1960-1969


In [34]:
import pandas as pd

def is_latin(s: str) -> bool:
    """
    Vérifie si la chaîne est écrite en alphabet latin.
    """
    try:
        s.encode('latin-1')
    except UnicodeEncodeError:
        return False
    else:
        return True


def get_title(df: pd.DataFrame, title_id: str) -> str:
    """
    Retourne le titre original si écrit en alphabet latin, sinon le titre string.
    """
    titre_id = df[df['titre_id'] == title_id]
    if titre_id.empty:
        return "ID de titre non trouvé"

    titre_og = titre_id.iloc[0]['original_title']
    if is_latin(titre_og):
        return titre_og
    else:
        return titre_id.iloc[0]['titre_str']

titre_id = 'tt9900782'
print(get_title(tmd1, titre_id))


Kaithi


In [2]:
config = import_config()
datas = GetDataframes(config)
df = datas.get_machine_learning_dataframe(
    "machine_learning",
    # modify=True
)

2023-11-11 09:40:59 INFO     Parquet loaded ! Importing machine_learning...
2023-11-11 09:41:00 INFO     Dataframe machine_learning ready to use!


In [8]:
df

,index,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors,overview
0,0,tt0035423,Kate & Leopold,"Comedy, Fantasy, Romance",6.40,87897,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",James Mangold,scientist ex boyfriend discovers portal travel...
1,1,tt0052832,The Fugitive Kind,"Drama, Romance",7.10,6914,"Anna Magnani, Marlon Brando, Joanne Woodward, ...",Sidney Lumet,val xavier drifter obscure origin arrives smal...
2,2,tt0052997,The League of Gentlemen,"Comedy, Crime, Drama",7.20,5105,"Jack Hawkins, Roger Livesey, Nigel Patrick, Ri...",Basil Dearden,involuntarily retired colonel hyde recruit sev...
3,3,tt0053459,Eyes Without a Face,"Drama, Horror",7.60,34278,"Pierre Brasseur, Alexandre Rignault, Alida Val...",Georges Franju,dr g nessier riddled guilt accident caused dis...
4,4,tt0053579,Late Autumn,"Comedy, Drama",7.90,5854,"Setsuko Hara, Keiji Sada, Mariko Okada, Yôko T...",Yasujirô Ozu,woman daughter forced contend increasing press...
...,...,...,...,...,...,...,...,...,...
7649,7651,tt9873892,They Cloned Tyrone,"Comedy, Mystery, Sci-Fi",6.60,33197,"Kiefer Sutherland, Jamie Foxx, John Boyega, Te...",Juel Taylor,series eerie event thrust unlikely trio onto t...
7650,7652,tt9883996,Dream Horse,"Biography, Comedy, Drama",6.90,6589,"Alan David, Lynda Baron, Owen Teale, Toni Coll...",Euros Lyn,inspiring true story dream alliance unlikely r...
7651,7653,tt9893250,I Care a Lot,"Comedy, Crime, Drama",6.40,141372,"Dianne Wiest, Rosamund Pike, Peter Dinklage, E...",J Blakeson,court appointed legal guardian defrauds older ...
7652,7654,tt9900782,Kaithi,"Action, Adventure, Crime",8.50,39546,"Narain, Karthi, George Maryan, Arjun Das",Lokesh Kanagaraj,dilli convicted criminal parole meet daughter ...


In [24]:
df[df["actors"].str.contains("Funès")]

,index,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors,overview
182,182,tt0057967,The Sucker,"Adventure, Comedy, Crime",7.30,7800,"Bourvil, Louis de Funès, Henri Génès, Venantin...",Gérard Oury,frenco italian gangster parody shop keeper way...
188,188,tt0058089,Fantomas,"Adventure, Comedy, Crime",6.90,11609,"Jean Marais, Louis de Funès, Jacques Dynam, My...",André Hunebelle,fant ma man many disguise us maquillage weapon...
191,191,tt0058135,The Gendarme of Saint-Tropez,"Comedy, Crime",7.10,13464,"Louis de Funès, Michel Galabru, Geneviève Grad...",Jean Girault,ambitious police officer cruchot transferred s...
252,252,tt0059168,Fantomas Unleashed,"Adventure, Comedy, Crime",6.80,7948,"Jean Marais, Louis de Funès, Jacques Dynam, My...","André Hunebelle, Haroun Tazieff",second episode trilogy fant ma kidnaps disting...
305,305,tt0060400,Fantomas vs. Scotland Yard,"Adventure, Comedy, Crime",6.70,7589,"Jean Marais, Louis de Funès, Françoise Christo...",André Hunebelle,third final episode trilogy fant ma imposes he...
310,310,tt0060450,The Gendarme in New York,"Comedy, Crime",6.40,8156,"Louis de Funès, Michel Galabru, Christian Mari...",Jean Girault,second installment gendarme series tell story ...
313,313,tt0060474,La Grande Vadrouille,"Adventure, Comedy, War",7.90,19551,"Bourvil, Louis de Funès, Claudio Brook, Andréa...",Gérard Oury,world war ii two french civilian downed britis...
359,359,tt0061728,What's Cooking in Paris,"Action, Comedy",6.70,6389,"Bernard Blier, Louis de Funès, Venantino Venan...",Jacques Besnard,great french restaurant owner monsieur septime...
373,373,tt0062083,Oscar,Comedy,7.50,7333,"Louis de Funès, Claude Rich, Mario David, Germ...",Édouard Molinaro,film originated play paris story focus one day...
405,405,tt0063005,The Gendarme Gets Married,"Comedy, Romance",6.80,8356,"Louis de Funès, Jean Lefebvre, Geneviève Grad,...",Jean Girault,saint tropez police launch major offensive dan...


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

df_movies = df.copy()
# Préparation des données pour le modèle
# Combinaison des colonnes 'titre_genres', 'actors', 'directors', et 'overview'
df_movies['combined_features'] = df_movies['titre_genres'] + ' ' + df_movies['actors'] + ' ' + df_movies['directors'] + ' ' + df_movies['overview']

# Vectorisation des caractéristiques combinées
vectorizer = TfidfVectorizer(stop_words='english')
feature_vectors = vectorizer.fit_transform(df_movies['combined_features'])

# Calcul de la similarité cosinus entre les films
cosine_sim = cosine_similarity(feature_vectors)

# Fonction pour obtenir les recommandations
def get_recommendations(title, num_recommendations=5):
    # Trouver l'index du film donné
    idx = df_movies[df_movies['titre_str'] == title].index[0]

    # Obtenir les scores de similarité pour ce film avec tous les autres
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Trier les films en fonction des scores de similarité
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtenir les indices des films les plus similaires
    sim_scores = sim_scores[1:num_recommendations+1]

    # Obtenir les titres des films recommandés
    movie_indices = [i[0] for i in sim_scores]
    return df_movies['titre_str'].iloc[movie_indices]

# Testons la fonction avec le film "Avatar" et 5 recommandations
recommended_movies = get_recommendations('Fight Club', 5)
recommended_movies


3859    The Curious Case of Benjamin Button
4056                            Rescue Dawn
2192                    Legends of the Fall
3957                        The Illusionist
3550                          The Machinist
Name: titre_str, dtype: object

In [11]:
from sklearn.neighbors import NearestNeighbors

# Utilisation de Nearest Neighbors pour les recommandations de films
# On utilise les mêmes vecteurs de caractéristiques créés précédemment
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6, n_jobs=-1)
model_knn.fit(feature_vectors)

# Fonction pour obtenir les recommandations avec Nearest Neighbors
def get_recommendations_knn(title, num_recommendations=5):
    # Trouver l'index du film donné
    idx = df_movies[df_movies['titre_str'] == title].index[0]

    # Recherche des voisins les plus proches
    distances, indices = model_knn.kneighbors(feature_vectors[idx], n_neighbors=num_recommendations+1)

    # Obtenir les titres des films recommandés
    movie_indices = [df_movies['titre_str'].iloc[i] for i in indices.flatten()[1:]]
    return movie_indices

# Testons la fonction avec le film "Avatar" et 5 recommandations
recommended_movies_knn = get_recommendations_knn('Fight Club', 5)
recommended_movies_knn


['The Curious Case of Benjamin Button',
 'Rescue Dawn',
 'Legends of the Fall',
 'The Illusionist',
 'The Machinist']

In [10]:
# Ajout des colonnes 'rating_avg' et 'rating_votes' aux caractéristiques
df_movies['features_with_ratings'] = df_movies['combined_features'] + ' ' + df_movies['rating_avg'].astype(str) + ' ' + df_movies['rating_votes'].astype(str)

# Vectorisation des nouvelles caractéristiques
feature_vectors_with_ratings = vectorizer.fit_transform(df_movies['features_with_ratings'])

# Entraînement du modèle Nearest Neighbors avec les nouvelles caractéristiques
model_knn_with_ratings = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6, n_jobs=-1)
model_knn_with_ratings.fit(feature_vectors_with_ratings)

# Test des recommandations avec les nouvelles caractéristiques pour le film "Avatar"
recommended_movies_with_ratings = get_recommendations_knn('Fight Club', 5)
recommended_movies_with_ratings


['The Curious Case of Benjamin Button',
 'Rescue Dawn',
 'Legends of the Fall',
 'The Illusionist',
 'The Machinist']

In [3]:
# df.to_csv("df_for_marchine_learning.csv", index=False)

In [4]:
def full_lower(
    text: str
):
    return text.replace(" ", "").replace("-", "")

tt = (
    # ("titre_genres", "titre_genres"),
    ("actors", "actors"),
    ("titre_genres", "titre_genres"),
    ("directors", "directors"),
)
for t in tt:
    df[t[0]] = df[t[1]].apply(full_lower)


# df["overview"] = df["overview"].apply(lambda x : x.replace(" ", ","))


In [5]:
print(df[df["titre_str"].str.contains("The Avengers")].to_markdown())

|      |   index | titre_id   | titre_str    | titre_genres   |   rating_avg |   rating_votes | actors                                                    | directors   | overview                                                                                                                                                                                                                |
|-----:|--------:|:-----------|:-------------|:---------------|-------------:|---------------:|:----------------------------------------------------------|:------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 4330 |    4330 | tt0848228  | The Avengers | Action,SciFi   |            8 |        1437952 | RobertDowneyJr.,ScarlettJohansson,ChrisEvans,JeremyRenner | JossWhedon  | unexpected enemy emerges threatens global safety secur

In [6]:
print(df[df["titre_id"].str.contains("tt2395417")].to_markdown())


|      |   index | titre_id   | titre_str   | titre_genres   |   rating_avg |   rating_votes | actors                                             | directors      | overview                                           |
|-----:|--------:|:-----------|:------------|:---------------|-------------:|---------------:|:---------------------------------------------------|:---------------|:---------------------------------------------------|
| 6147 |    6148 | tt2395417  | Still Life  | Drama          |          7.4 |           8708 | KarenDrury,EddieMarsan,JoanneFroggatt,AndrewBuchan | UbertoPasolini | council case worker look relative found dead alone |


In [7]:
def combine(r):
    return r["titre_genres"]+" "+r["directors"]+" "+r["overview"]+" "+r["actors"] #r["titre_str"]+" "+

df["one_for_all"] = df.apply(combine, axis=1)

In [14]:
def titre_index(titre: str):
    return df[df.titre_str == titre].index[0]

def director_index(director: str):
    return df[df.directors.str.contains(director)].index[0]

def actor_index(actor: str):
    return df[df.actors.str.contains(actor)].index[0]

def idx_titre(idx: int):
    return df[df.index == idx]["titre_str"].values[0]

def idx_actor(idx: int):
    return df[df.index == idx]["actors"].values[0]

def idx_titre_id(idx: int):
    return df[df.index == idx]["titre_id"].values[0]

In [15]:
# tidf = TfidfVectorizer()
# combined_matrix = tidf.fit_transform(
#     df["titre_str"]+" "+df["actors"]+" "+df["titre_genres"]+" "+df["directors"]+" "+df["overview"]
# )
# tidf = TfidfVectorizer()
# combined_matrix = tidf.fit_transform(
#     df["one_for_all"]+" "+df["titre_str"]
# )
top = 10

poids_ = {
    # "titre_str":    2,
    "titre_genres": 2,
    "actors":       1.5,
    "directors":    1.5,
    "overview":     2.5,
    # "rating_avg":   0.2,
    # "rating_votes": 0.2,
}

full_matrix = []
for col, poids in poids_.items():
    tfidf_ = TfidfVectorizer()
    matrix_ = tfidf_.fit_transform(df[col]) * poids
    full_matrix.append(matrix_)

combined_matrix = hstack(full_matrix)

cosine = cosine_similarity(combined_matrix)

movies = "Fight Club"
mov_idx = titre_index(movies)

similar = cosine[mov_idx]
similar1 = list(enumerate(cosine[mov_idx]))

sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)
sim_mov_idx = similar.argsort()[::-1][1:top+1]

same_movies = df.loc[sim_mov_idx, "titre_str"]
ttconst = df.loc[sim_mov_idx, "titre_id"]

sim_scores[1:top+1]
score = [i[1] for i in sim_scores]

print(f"Top 10 similar movies to {movies} are :\n")
for movies_, tt,  score in zip(same_movies, ttconst, score):
    print(f"Movie : {movies_} | id : {tt} | score : {np.round(score, 4)}")


Top 10 similar movies to Fight Club are :

Movie : Crash | id : tt0115964 | score : 1.0
Movie : Naked Lunch | id : tt0102511 | score : 0.3286
Movie : Combat Girls | id : tt1890373 | score : 0.3222
Movie : The Spitfire Grill | id : tt0117718 | score : 0.3196
Movie : George Washington | id : tt0262432 | score : 0.3142
Movie : Radio Flyer | id : tt0105211 | score : 0.3113
Movie : Troy | id : tt0332452 | score : 0.3076
Movie : Jacknife | id : tt0097607 | score : 0.3069
Movie : 25th Hour | id : tt0307901 | score : 0.305
Movie : After the Wedding | id : tt7985692 | score : 0.3026


In [21]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer

# movies = "Fight Club"
print(movies)

cv = CountVectorizer()
count_matrix = cv.fit_transform(df['one_for_all'])
movie_user_likes = movies
movie_index = titre_index(movie_user_likes)

knn_model = NearestNeighbors(metric='cosine', algorithm='auto')
knn_model.fit(count_matrix)
dist, indices = knn_model.kneighbors(count_matrix[movie_index], n_neighbors=top)

print(f"Top 10 similar movies to {movie_user_likes} are :\n")
for index, dis in zip(indices.flatten()[1:], dist.flatten()[1:]):
    print(f"Movie : {idx_titre(index)} | id : {idx_titre_id(index)} | score : {np.round(dis, 4)}")


Avatar
Top 10 similar movies to Avatar are :

Movie : Avatar: The Way of Water | id : tt1630029 | score : 0.7474
Movie : Predators | id : tt1424381 | score : 0.8127
Movie : Aliens | id : tt0090605 | score : 0.8214
Movie : Beowulf | id : tt0442933 | score : 0.8218
Movie : The Book of Life | id : tt2262227 | score : 0.8275
Movie : Valerian and the City of a Thousand Planets | id : tt2239822 | score : 0.8339
Movie : Venom | id : tt1270797 | score : 0.8344
Movie : The Extraordinary Adventures of Adèle Blanc-Sec | id : tt1179025 | score : 0.8367
Movie : Star Wars: Episode VIII - The Last Jedi | id : tt2527336 | score : 0.8367


['Avatar: The Way of Water',
 'Aliens',
 'The Book of Life',
 'Guardians of the Galaxy Vol. 2',
 'Galaxy Quest']

In [20]:

# random_model = RandomForestClassifier().fit(combined_matrix, df["titre_genres"])
# for idx, movies in zip(sim_mov_idx, same_movies):
#     mo = combined_matrix[idx].toarray()
#     prediction = random_model.predict(mo)[0]
#     confidence = np.max(random_model.predict_proba(mo))

#     print("movie :", movies)
#     print("confidence :", confidence)